# Start

In [1]:
import sys
from importlib import reload

import pandas as pd

import fraud_detection as fd

# Add the src directory to the system path to import modules
reload(fd)
sys.path.append('../src')

datapath = "../data/transformed_label_and_damage.parquet"

seed = 42

In [2]:
datapath = "../data/transformed_label_and_damage.parquet"

In [3]:
useless_features = [
    "max_product_price",
    "has_positive_price_difference",
    "has_bakery",
    "time_to_first_scan",
    "popularity_max",
    "has_age_restricted",
    "cash_desk",
    "transaction_duration_seconds",
    "feedback_low",
    "feedback_middle",
    "feedback_high",
    "feedback_top",
    "store_id",
    "location",
    "urbanization",
    "has_voided",
    "has_sold_by_weight",
    "has_limited_time_offers",
    "has_fruits_vegetables",
    "has_missing",
    "has_camera_detected_wrong_product",
    "day_of_week",
    "hour_categorical"
]

In [9]:
keep = [
"n_lines"
,"payment_medium"
,"has_feedback"
,"feedback_categorical"
,"hour"
,"transaction_duration_seconds"
,"has_voided"
,"has_camera_detected_wrong_product"
,"has_camera_detected_wrong_product_high_certainty"
,"calculated_price_difference"
,"has_positive_price_difference"
,"has_snacks"

]

# Model Vergleich

Vergleich verschiedener Modelle mit Cross Validition auf identischen Splits.

Die Combinded Modelle bestehen aus einem Klassifikator und einem Regressor, die zusammenarbeiten, um eine bessere Vorhersage zu erzielen.

In [10]:
%%capture

X, _ = fd.data_loader.load_data_np(datapath)
neural_net = fd.neuralnets.train_nn.getNN(X.shape[1])

models = [
    # ("NeuralNet", clf),
    fd.models.classifiers.get_lgmb(),
    fd.models.classifiers.get_xgb_simple(),
    fd.models.classifiers.get_catboost(),
    # fd.models.models.get_xgb_clf_with_reg(),
    # fd.models.get_xgb(),
    # ("XGBoost Plus", fd.models.models.get_xgb_clf_with_reg(10)),
]

model_metrics = fd.model_comparison.compare_models(
    models, datapath, n_splits=5, random_state=seed, select_features=keep#, drop_features=useless_features
)

In [11]:
# hour dropped

model_metrics_df = pd.DataFrame(model_metrics)
model_metrics_df.T

,precision,recall,f1,mcc,damage_total,damage_prevented,damage_missed,detected bonus,fp penalty,Bewertung,cm
LGMB Classifier,0.809318,0.499294,0.617556,0.627621,6015.164,3298.436,2716.728,2136.0,1006.0,-1586.728,"[[28573.2, 100.6], [428.4, 427.2]]"
XGB Simple Classifier,0.811592,0.504201,0.621894,0.63165,6015.164,3334.072,2681.092,2157.0,1002.0,-1526.092,"[[28573.6, 100.2], [424.2, 431.4]]"
CatBoost Classifier,0.823608,0.493218,0.616946,0.629512,6015.164,3287.542,2727.622,2110.0,904.0,-1521.622,"[[28583.4, 90.4], [433.6, 422.0]]"


In [ ]:
#h_dropped = model_metrics_df
h_dropped

,LGMB Classifier,XGB Simple Classifier,CatBoost Classifier
precision,0.843031,0.844653,0.860763
recall,0.55002,0.552119,0.543944
f1,0.665638,0.667702,0.666534
mcc,0.67364,0.675644,0.677199
damage_total,6015.164,6015.164,6015.164
damage_prevented,3541.55,3561.514,3519.556
damage_missed,2473.614,2453.65,2495.608
detected bonus,2353.0,2362.0,2327.0
fp penalty,876.0,868.0,752.0
Bewertung,-996.614,-959.65,-920.608


In [12]:
a = [ "calculated_price_difference",
  "has_positive_price_difference",

  "day_of_week",
  "daytime",
  "days_since_sco_introduction",
  "hour",
  "hour_categorical",
  "month",

  "feedback_categorical",
  "feedback_high",
  "feedback_low",
  "feedback_middle",
  "feedback_top",
  "has_feedback",

  "has_camera_detected_wrong_product",
  "has_camera_detected_wrong_product_high_certainty",

  "has_alcohol",
  "has_bakery",
  "has_beverages",
  "has_convenience",
  "has_dairy",
  "has_frozen_goods",
  "has_fruits_vegetables",
  "has_fruits_vegetables_pieces",
  "has_household",
  "has_limited_time_offers",
  "has_long_shelf_life",
  "has_missing",
  "has_personal_care",
  "has_snacks",
  "has_tobacco",

  "has_voided",
  "has_sold_by_weight",
  "has_age_restricted",
  "n_age_restricted",
  "n_sold_by_weight",
  "n_voided",

  "max_product_price",
  "popularity_max",
  "popularity_min",

  "payment_medium",
  "total_amount",
  "n_lines",

  "transaction_duration_seconds",
  "mean_time_between_scans",
  "max_time_between_scans",
  "time_to_first_scan",
  "time_from_last_scan_to_end",

  "store_id",
  "cash_desk",
  "location",
  "urbanization"
]
len(a)

52

Das neuronale Netzwerk ist nocht recht unstabil und erzielt nicht immer gute Ergebnisse (in diesem Durchlauf schnitt es aber am besten ab).